In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Permute, Reshape, Input,Concatenate,concatenate,dot,Reshape,add,multiply,maximum, Dot, Add, multiply
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Conv1D, GlobalMaxPool2D, GlobalAveragePooling1D, Lambda, Input, Layer
from keras.layers.convolutional import ZeroPadding2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.models import Model
import tensorflow as tf
from keras.layers.merge import _Merge

import numpy as np
import pandas as pd
import random

Using TensorFlow backend.
/home/adminis/anaconda3/envs/py36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val.append(logs.items())

# Load processed data

In [12]:
dataset = 'UCL' #PAMAP2
set = 'test' #train
modalities = 6 #18 for PAMAP2
values = 128
channels = 1
nb_classes = 6 #12 for PAMAP2

s_c = np.load(set+'_'+dataset+'_data.npy')
s_q = np.load(set+'_'+dataset+'_questions.npy')
y = np.load(set+'_'+dataset+'_labels.npy')

# Matching Network

In [35]:
def perceptionNet(input_shape):

    input_acc = Input(shape=(modalities, values, channels))
    x = Conv2D(32, (1, 11), padding='same')(input_acc)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(1, 3))(x)
    x = Dropout(0.50)(x)

    z = Conv2D(48, (1, 11), padding='same')(x)
    z = Activation('relu')(z)
    z = MaxPooling2D(pool_size=(1, 3))(z)
    z = Dropout(0.50)(z)
    
    y = Conv2D(64, (3, 11), padding='same')(z)
    y = Activation('relu')(y)
    y = MaxPooling2D(pool_size=(1, 2))(y)
    y = Dropout(0.5)(y)
    y = Flatten()(y)

    return Model(input_acc,y)

In [36]:
def cosine_similarity(vects):
    x, y = vects
    eps = 1e-10
    
    sum_support = K.sum(K.square(y), 1, keepdims=True)
    supportmagnitude = tf.rsqrt(tf.clip_by_value(sum_support, eps, float("inf")))
    
    sum_query = K.sum(K.square(x), 1, keepdims=True)
    querymagnitude = tf.rsqrt(tf.clip_by_value(sum_support, eps, float("inf")))
    
    x_ = K.expand_dims(x,1)
    y_ = K.expand_dims(y,2)
    
    dot_product = K.batch_dot(x_,y_)

    dot_product = K.squeeze(dot_product,1)

    
    cosine_sim = dot_product*supportmagnitude#*querymagnitude
    return cosine_sim

In [37]:
def matchNet():
    similarities = []

    # network definition
    input_shape = (modalities, values, channels)
    perception = perceptionNet(input_shape)

    input_a = Input(shape=input_shape)
    processed_a = perception(input_a)

    input_b = Input(shape=(nb_classes, modalities, values, channels))

    modelinputs = []
    for lidx in range(nb_classes):
        modelinputs.append(perception(Lambda(lambda x: x[:,lidx,:,:,:])(input_b)))

    for i in range(nb_classes):

        sim = Lambda(cosine_similarity)([processed_a, modelinputs[i]])
        similarities.append(sim)

    similarities = Concatenate(axis=1)(similarities)
    soft = Activation('softmax')(similarities)
    
    model = Model([input_a, input_b], soft)
    return model

In [38]:
accs_t = []

for i in range(0,10):
    model = matchNet()
    model.load_weights('trained_models/matchnet_UCL_'+str(i)+'.h5')
    acc = np.mean(np.argmax(model.predict([s_q,s_c]),1)==y)
    accs_t.append(acc)
print(np.mean(accs_t))

0.9303827751196172


# Relational Network

In [16]:
def perceptionNet(input_shape):

    input_acc = Input(shape=(modalities, values, channels))
    x = Conv2D(32, (1, 11), padding='same')(input_acc)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(1, 3))(x)
    x = Dropout(0.50)(x)

    z = Conv2D(48, (1, 11), padding='same')(x)
    z = Activation('relu')(z)
    z = MaxPooling2D(pool_size=(1, 3))(z)
    z = Dropout(0.50)(z)
    
    y = Conv2D(64, (1, 11), padding='same')(z)
    y = Activation('relu')(y)
    y = MaxPooling2D(pool_size=(1, 2))(y)

    return Model(input_acc,y)

In [27]:
class MultiheadDot(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head * size_per_head
        self.__name__ = 'MultiheadDot'
        super(MultiheadDot, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_normal',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_normal',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_normal',
                                  trainable=True)
        
        self.WA = self.add_weight(name='WA', 
                                  shape=(self.nb_head*self.size_per_head, 64),
                                  initializer='glorot_normal',
                                  trainable=True)
        self.WE = self.add_weight(name='WE', 
                                  shape=(64, 1),
                                  initializer='glorot_normal',
                                  trainable=True)
        
        super(MultiheadDot, self).build(input_shape)

    def call(self, x):
        Q_in,K_in,V_in = x
        
        Q = K.dot(Q_in, self.WQ)
        Q = K.reshape(Q, (-1, 6, 7, self.nb_head, self.size_per_head))
        Q = K.permute_dimensions(Q, (0,3,1,2,4))
        
        Key = K.dot(K_in, self.WK)
        Key = K.reshape(Key, (-1, 6, 7, self.nb_head, self.size_per_head))
        Key = K.permute_dimensions(Key, (0,3,1,2,4))
        
        V = K.dot(V_in, self.WV)
        V = K.reshape(V, (-1, 6, 7, self.nb_head, self.size_per_head))
        V = K.permute_dimensions(V, (0,3,1,2,4))
        
        A = tf.keras.backend.batch_dot(Q, Key, axes=[4,4]) / self.size_per_head**0.5
        A = K.softmax(A)
        
        E = tf.keras.backend.batch_dot(A, V, axes=[4,3])
        E = K.permute_dimensions(E, (0,2,3,1,4))
        E = K.reshape(E, (-1, 6*7, self.output_dim))
        
        sim = K.dot(E, self.WA)
        sim = K.relu(sim)
        sim = K.dropout(sim, 0.5)
        sim = K.mean(sim, 1)
        sim = K.dot(sim, self.WE)
        
        return [sim, A]
    
    def compute_output_shape(self, input_shape):
        return [(input_shape[0][0], 1),(input_shape[0][0], 2, 6, 7, 7)]
    
    def get_config(self):
        config = super(MultiheadDot, self).get_config()
        config.update({"nb_head": self.nb_head, "size_per_head":self.size_per_head})
        return config

In [3]:
nb_head = 3 # 2 for PAMAP2
size_per_head = 64 # d variable

In [32]:
# network definition
def relationalModule():
    similarities = []
    attentions = []
    input_shape = (modalities, values, channels)

    perception = perceptionNet(input_shape)
    multihead = MultiheadDot(nb_head, size_per_head)
    
    config = multihead.get_config()
    new_multihead = MultiheadDot.from_config(config)

    input_a = Input(shape=input_shape)
    processed_a = perception(input_a)
    flatten_a = Flatten()(processed_a)

    input_b = Input(shape=(nb_classes, modalities, values, channels))

    modelinputs = []
    for lidx in range(nb_classes):
        modelinputs.append(perception(Lambda(lambda x: x[:,lidx,:,:,:])(input_b)))

    for i in range(nb_classes):
        att, ma = new_multihead([processed_a, modelinputs[i], modelinputs[i]])
        similarities.append(att)
        attentions.append(ma)

    similarities = Concatenate(axis=1)(similarities)
    soft = Activation('softmax')(similarities)
    
    model = Model([input_a, input_b], soft)
    
    model_att = Model([input_a, input_b], attentions)
    
    return model, model_att

In [33]:
accs_t = []

for i in range(0,10):
    
    model,_ = relationalModule()
    
    model.load_weights('trained_models/relational_UCL_'+nb_head+'_'+size_per_head+'_'+str(i)+'.h5')
    acc = np.mean(np.argmax(model.predict([s_q,s_c]),1)==y)
    accs_t.append(acc)
print(np.mean(accs_t))

0.9480861244019139
